In [31]:
import pandas as pd

#df = pd.read_csv('../data/donnees_nettoyees_avec_coord.csv', index_col=0)

df = pd.read_pickle('../data/dataframe_clean_last_version.pkl')
print(len(df), " individus")


print(df['Departement'].nunique()," départements différents")
df.head(15)



2370400  individus
95  départements différents


,No_disposition,Nature_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,...,Nature_culture,Surface_terrain,day,month,year,latitude_x,longitude_x,latitude_y,longitude_y,Departement
0,1,Vente,166250.0,12290,CANET DE SALARS,C,479,0,1,54.0,...,T,2251.0,13,7,2018,44.232587,2.737756,44.232587,2.737756,12
1,1,Vente,14000.0,81440,LAUTREC,D,342,0,3,0.0,...,S,29.0,14,9,2018,43.705247,2.129294,43.705247,2.129294,81
2,1,Vente,235000.0,95500,GONESSE,AI,52,0,1,74.0,...,S,396.0,21,12,2018,48.986490,2.456990,48.986490,2.456990,95
3,1,Vente,87000.0,36120,JEU LES BOIS,B,656,0,1,60.0,...,S,1100.0,21,11,2018,46.681894,1.794628,46.681894,1.794628,36
4,1,Vente,590000.0,33800,BORDEAUX,EO,192,0,1,110.0,...,S,313.0,6,6,2018,44.857245,-0.573697,44.857245,-0.573697,33
5,1,Vente,158000.0,29270,CARHAIX PLOUGUER,AB,198,0,1,132.0,...,S,500.0,30,7,2018,48.268758,-3.566264,48.268758,-3.566264,29
6,1,Vente,320000.0,95450,US,AH,275,0,1,112.0,...,S,605.0,28,8,2018,49.104808,1.957646,49.104808,1.957646,95
7,1,Vente,326350.0,54210,VILLE EN VERMOIS,G,388,0,1,145.0,...,P,368.0,26,6,2018,48.619488,6.244289,48.619488,6.244289,54
8,1,Vente,132300.0,17430,TONNAY CHARENTE,AH,37,0,1,53.0,...,S,364.0,24,1,2018,45.963377,-0.874412,45.963377,-0.874412,17
9,1,Vente,420000.0,34090,MONTPELLIER,AW,28,0,4,500.0,...,S,489.0,15,1,2018,43.613441,3.868517,43.613441,3.868517,34


In [27]:
import folium
from folium.plugins import HeatMap
from branca.colormap import linear

# Créez une liste de listes contenant les coordonnées [latitude, longitude] et la valeur "Valeur fonciere"
heat_data = [[row['latitude'], row['longitude'], row['Valeur_fonciere']]
             for index, row in df.iterrows()]

# Créez la carte
my_map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Créez une colormap linéaire basée sur la colonne "Valeur fonciere"
colormap = linear.YlOrRd_09.scale(
    min(df['Valeur_fonciere']), max(df['Valeur_fonciere']))

# Ajoutez la carte de chaleur à partir des données
HeatMap(heat_data,
        min_opacity=0.2,
        max_val=max(df['Valeur_fonciere']),
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(my_map)

# Ajoutez une légende basée sur la colormap
colormap.caption = "Valeur fonciere"
colormap.add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map.html')


C:\Users\nagrimault\AppData\Local\Temp\ipykernel_11880\449697079.py:17: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data,


In [28]:
import folium
from folium.plugins import HeatMap
from folium import Choropleth, GeoJson
import pandas as pd

# Calcul de la valeur foncière moyenne par département
df_avg_value_by_dept = df.groupby(
    'Departement')['Valeur_fonciere'].mean().reset_index()

print(df_avg_value_by_dept)
print(len(df_avg_value_by_dept), " départements différents")
# Coordonnées du centre de la France (ou un autre emplacement de votre choix)
latitude, longitude = 46.603354, 1.888334

# Créez la carte
my_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajoutez la carte choroplèthe en fonction de la valeur foncière moyenne par département
Choropleth(
    # Chemin vers le fichier GeoJSON des départements de la France
    geo_data='../data/departements.geojson',
    name='choropleth',
    data=df_avg_value_by_dept,
    columns=['Departement', 'Valeur_fonciere'],
    # Clé pour faire correspondre les données GeoJSON avec le DataFrame
    key_on='feature.properties.code',
    # Palette de couleurs (vous pouvez choisir d'autres palettes)
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Valeur fonciere moyenne par département'
).add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map_departements.html')


   Departement  Valeur_fonciere
0           01     3.177374e+05
1           02     2.415631e+05
2           03     4.244532e+05
3           04     2.232140e+05
4           05     2.523776e+05
..         ...              ...
90          92     2.415628e+06
91          93     6.676503e+05
92          94     9.368471e+05
93          95     8.082404e+05
94          97     4.255839e+05

[95 rows x 2 columns]
95  départements différents


In [44]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_geocoder")

# Remplacez le code postal correct de la ville voulue et vérifier les coordonnées
code_postal = "2200"
adresse = f"SOISSONS, {code_postal}, France"

# Utilisez Nominatim pour obtenir les coordonnées géographiques
location = geolocator.geocode(adresse)

# Affichez les coordonnées géographiques (latitude et longitude) de "AVERMES"
if location:
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Coordonnées non trouvées pour l'adresse spécifiée.")


Latitude: 49.3837621, Longitude: 3.327613
